In [2]:
import tensorflow as tf
from tensorflow.keras.preprocessing import image
import numpy as np
import cv2
print(tf.__version__)

1.10.0


In [3]:
loaded_model = tf.keras.models.load_model("F:/Study Materials/Summer 2020/Mask_recg/model_just_load.h5")
#Copyright (C) 2000, Intel Corporation, all rights reserved.
# Third party copyrights are property of their respective owners.
face_cascade = cv2.CascadeClassifier("haarcascade_frontalface_default.xml")

In [5]:
cap = cv2.VideoCapture(0)                                           # 0 stands for webcam 

#fourcc = cv2.VideoWriter_fourcc(*'XVID')
#out = cv2.VideoWriter("Mask_rec.mp4",fourcc,20.0,(640,480)) 

#print(cap.get(cv2.CAP_PROP_FRAME_WIDTH))                            # get and print Frame Width
#print(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))                           # get and print Frame Height

while(cap.isOpened()):                                              # if we can capture the video then True
    ret, frame = cap.read()                                         # read the capture and store in Frame
    if ret:
        
        
        grey = cv2.cvtColor(frame,cv2.COLOR_BGR2GRAY)               # convert the frame to greyscale
        faces = face_cascade.detectMultiScale(grey, 1.1, 4)         # detect the faces from the greyscale frame

        for (x, y, w, h) in faces:
            
            face = frame[y:y+h,x:x+w]                               # get the face image 
            face = cv2.resize(face,(150,150))                       # resize the face image to (150,150)
            
            face=image.img_to_array(face)                           # convert the image to array
            face=np.expand_dims(face,axis=0)                        # expand the dim of the array
            images = np.vstack([face])                              # stack the sequence of input array vertically
            
            classes = loaded_model.predict(images)                  # predict the class(i.e. mask or no mask) using loaded model 
            
            font = cv2.FONT_HERSHEY_SIMPLEX                                      
            if classes==1:
                rect = cv2.rectangle(frame, (x,y), (x+w,y+h), (255,255,255), 3)       # draw the rectangle
                cv2.putText(rect,"Mask",(x,y-10), font, 0.75,(255,255,255),1)         # print text on the video  
 #               out.write(frame)                                                  # save the output
            else:
                rect = cv2.rectangle(frame, (x,y), (x+w,y+h), (0,0,255), 3)
                cv2.putText(rect,"No Mask",(x,y-10), font, 0.75,(0,0,255),1)
 #               out.write(frame)
                
        cv2.imshow("Face_detect",rect)                              # show the frame
        
        if cv2.waitKey(1) & 0xFF == ord('q'):                       # if q is pressed break(Case Sensitive)
            break
    else:
        break
        
cap.release()                                                       # release the captured video
#out.release()
cv2.destroyAllWindows()                                             # destroy all windows